In [1]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [17]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

### Pre-filtering based on inventory ###
# Search based on specific filters
page = 9
page_size = 1
craft = 'crochet'
knit_gauge = 5
weight = 'DK'
query = 'bear'

# Define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?page={}&page_size={}&craft={}'.format(page, page_size, craft)
# Make request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
# Close connection
r.close()

def getPatterns():
  yarndf = pd.DataFrame()
  keydf = pd.DataFrame()
  if r.status_code == 200:
      data = r.json()
      # Extract pattern ID(s) from the search results
      if 'patterns' in data and len(data['patterns']) > 0:
        for i in range(len(data['patterns'])):
            pattern_id = data['patterns'][i]['id']

            # Define URL to get pattern details
            pattern_url = f'https://api.ravelry.com/patterns/{pattern_id}.json'

            # Make the request for pattern details
            pattern_response = requests.get(pattern_url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
            pattern_response.close()

            # If patterns found, collect the necessary yarn info
            if pattern_response.status_code == 200:
                pattern_data = pattern_response.json()
                try:
                  print(json.dumps(json.loads(pattern_response.text), indent=4))
                  yarnData = pattern_data['pattern']['yarn_weight']

                  # Extract yardage for minimum/maximums
                  yardage = ['0', '0']
                  extracted = re.findall(r'\d+', pattern_data['pattern']['yardage_description'])

                  if len(extracted) == 2:
                    yardage = extracted
                  elif len(extracted) == 1:
                    yardage[0] = extracted[0]
                    yardage[1] = extracted[0]

                  # Get project name and add all to dataframe
                  yarndf = pd.concat([yarndf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                            'Yarn id': yarnData['id'],
                                                            'Crochet gauge': yarnData['crochet_gauge'],
                                                            'Knit gauge': yarnData['knit_gauge'],
                                                            'Yarn name': yarnData['name'],
                                                            'ply': yarnData['ply'], 'wpi': yarnData['wpi'],
                                                            'Min yardage': int(yardage[0]),
                                                            'Max yardage': int(yardage[1])}])], ignore_index=True)

                  # Get key information from pattern details for recommender
                  keydf = pd.concat([keydf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                          'Project id': pattern_data['pattern']['id'],
                                                          'Difficulty average': pattern_data['pattern']['difficulty_average'],
                                                          'UK': pattern_data['pattern']['has_uk_terminology'],
                                                          'US' : pattern_data['pattern']['has_us_terminology']}])], ignore_index=True)
                except:
                  print("No yarn weight recorded")
      else:
        print("No patterns found")
  else:
    print("Unable to access patterns")

  keydf = keydf.merge(yarndf, on='Project name')
  return keydf


def fetchInventory():
  # Open stored inventory into dataframe
  with open("inventory.json", "r") as file:
    inventory = json.load(file)
    inv = pd.DataFrame(inventory)
    groupedInv = inv.groupby(['Yarn name','ply', 'wpi', 'Total yardage']).agg(
        min_yardage=('Total yardage', 'min'),
        max_yardage=('Total yardage', 'max')).reset_index()

    print(groupedInv)
    return groupedInv

def filterPatterns(groupedInv, patterns):
  # Filters patterns for yarn available in inventory
  filtereddf = patterns[patterns['Yarn name'].isin(groupedInv['Yarn name'])]
  finaldf = pd.DataFrame(columns = patterns.columns)
  for idx, row in filtereddf.iterrows():
    rowYardage = row['Max yardage']
    maxYardage = int(max(groupedInv.loc[groupedInv['Yarn name'] == row['Yarn name'], 'max_yardage'].values))

    if (rowYardage > 0) & (rowYardage <= maxYardage):
      # Directly adds if empty for future compatibility
      if finaldf.empty:
        finaldf = pd.DataFrame([row])
      else:
        finaldf = pd.concat([finaldf, pd.DataFrame([row])], ignore_index = True)

  print(finaldf)

patterns = getPatterns()


{
    "pattern": {
        "comments_count": 0,
        "created_at": "2025/02/22 11:45:09 -0500",
        "currency": "CAD",
        "difficulty_average": 0,
        "difficulty_count": null,
        "downloadable": true,
        "favorites_count": 12,
        "free": false,
        "gauge": null,
        "gauge_divisor": 4,
        "gauge_pattern": "Any gauge!",
        "generally_available": "2025/02/22 11:45:09 -0500",
        "has_uk_terminology": null,
        "has_us_terminology": true,
        "id": 7410494,
        "name": "Candy Crush Pullover",
        "pdf_url": "",
        "permalink": "candy-crush-pullover",
        "price": 8.99,
        "projects_count": 4,
        "published": "2025/02/01",
        "queued_projects_count": 0,
        "rating_average": 0,
        "rating_count": null,
        "row_gauge": null,
        "updated_at": "2025/02/22 12:32:32 -0500",
        "url": "https://crochetwithcookie.com/products/candy-crush-pullover-crochet-pattern-pdf",
        "yar

In [3]:
groupedInv = fetchInventory()
filterPatterns(groupedInv, patterns)

   Yarn name  ply  wpi  Total yardage  min_yardage  max_yardage
0         DK    8   11            500          500          500
1         DK    8   12            200          200          200
2  Fingering    4   14            800          800          800
                   Project name  Project id  Difficulty average     UK     US  \
0                 Afternoon Tea     7352835            5.428571  False   True   
1                 Karma Coaster     7386414            2.000000   None   True   
2   Luna the Labrador Retriever     7403782            0.000000   None   True   
3           Valentine Love Hugs     7406457            2.666667   True   True   
4     Easter Mini Egg Pouch Bag     7406569            0.000000   True  False   
5            Love is in the Air     7407698            0.000000  False   True   
6          Pew the baby penguin     1091842            2.571429  False   True   
7              Carnation Flower     1224766            2.052632  False   True   
8    Offensive 

In [5]:
### Basic recommender ###
# Take the last 5 patterns saved by the user and use the average calculated metrics to then recommend
# In release, should use the saved and completed patterns

def readSaved():
  with open("saved.json", "r") as file:
    savedPatterns = json.load(file)
    saved = pd.DataFrame(savedPatterns)


